In [2]:
# 导入好样本
import pandas as pd
import gc
import numpy as np
table_1 = pd.read_table('open_data_train_valid/train/train_1.txt',sep = '\t')
columns = table_1.columns
table_2 = pd.read_table('open_data_train_valid/train/train_2.txt',sep = '\t',names= columns)
data = pd.concat([table_1,table_2],axis=0,ignore_index = True)
label = data['label']
label = label.iloc[label[label.notnull()].index.values]
data = data.iloc[label[label.notnull()].index.values]
X_good = data
del data
del table_1
del table_2
gc.collect()
X_good.drop(['id','loan_dt','label','tag'],axis=1,inplace = True)
#可供选择
#X_good, label

71

In [3]:
# 导入坏样本
table_1 = pd.read_table('open_data_train_valid/train/train_3.txt',sep = '\t',names=columns)
label_  = table_1['label']
table_1 = table_1.iloc[label_[label_.isnull()].index.values]
X_bad = table_1
del table_1
gc.collect()

#可供选择
#X_bad

21

In [7]:
# 选出特征
numerical_feature = np.load('open_data_train_valid/float64.npy')
factor_feature   = np.load('open_data_train_valid/int64.npy')

X_good_num = X_good[numerical_feature]
X_good_factor = X_good[factor_feature]

#X_bad_num = X_bad[numerical_feature]
#X_bad_factor = X_bad[factor_feature]

#X_num = pd.concat([X_good_num,X_bad_num],axis=0,ignore_index=False)
#X_factor = pd.concat([X_good_factor,X_bad_factor],axis=0,ignore_index=False)

#可供选择
#numerical_feature,factor_feature,X_good_num,X_good_factor
#X_bad_num,X_bad_factor,X_num,X_factor

In [10]:
# 组合做归一化
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#scaling numerical data
X_good_num.fillna(X_good_num.mean(axis=0),axis=0,inplace=True)
#X_bad_num.fillna(0,inplace=True)
#X_num.fillna(0,inplace=True)

scaler = StandardScaler()
#scaler.fit(X_num)
scaler.fit(X_good_num)
#X_num_fit = scaler.transform(X_num)
#X_good_num_fit = scaler.transform(X_good_num)
#X_bad_num_fit = scaler.transform(X_bad_num)
X_good_num = scaler.transform(X_good_num)

X_good = np.concatenate([X_good_num,X_good_factor.values],axis=1)
#X_num = pd.DataFrame(X_num_fit,index = X_num.index,columns=X_num.columns)
#X_good_num = pd.DataFrame(X_good_num_fit,index = X_good_num.index,columns=X_good_num.columns)
#X_bad_num = pd.DataFrame(X_bad_num_fit,index = X_bad_num.index,columns=X_bad_num.columns)

#columns = X_num.columns

#del X_num_fit
#del X_good_num_fit
#del X_bad_num_fit
#gc.collect()

#可供选择
#X_num,X_good_num,X_bad_num_fit,columns(updated)

In [ ]:
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier(
learning_rate = 0.1,
n_estimator = 150,
max_depth = 3,
min_child_weight = 1,
gamma = 0.1,
reg_alpha = 0.1,
reg_lambda = 3,
subsample = 0.8,
colsample_bytree = 0.8,
objective = 'binary:logistic',
nthread= 4,
scale_pos_weight = 15,
silent = False
)
xgb.fit(X_good,np.array(label), eval_metric = 'auc')

[22:52:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[22:52:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[22:53:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[22:53:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[22:53:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[22:53:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[22:54:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[22:54:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[22:54:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_

In [15]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
y_good_pred = xgb.predict(X_good)
auc = roc_auc_score(y_true = np.array(label),y_score = y_good_pred)
print(auc)

//anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8591403364831588


In [80]:
# 做第一步训练
#好样本分出训练、测试集
import xgboost as xgb
#X_good = pd.concat([X_good_num,X_good_factor],axis=1)
#X_bad = pd.concat([X_bad_num,X_bad_factor],axis=1)

#X_good_train, X_good_test, y_good_train, y_good_test = train_test_split(X_good,label,
#                                                                        test_size = 0.3,random_state = None)

params_good = {
    'booster' : 'gbtree',
    'objective': 'binary:logistic',
    'gamma': 0.1, #minimum loss reduction required to make a further partition on a leaf node of the tree
    'alpha':0.5,
    'max_depth': 4,
    'subsample' : 0.8, #proportion of train set used to train
    'colsample_tree' : 0.8, #proportion of sample of features
    'min_child_weight': 0.8,
    'silent': 0, #print process
    'eta' : 0.1, # step size
    'scale_pos_weight': 15
}
plst_good = params_good.items()

dX_good_train = xgb.DMatrix(X_good_train)
dgood_train = xgb.DMatrix(X_good_train,y_good_train)
num_rounds = 100
model_good = xgb.train(plst_good,dgood_train,num_rounds)

#可供选择
#X_good,X_bad,X_good_train, X_good_test, y_good_train, y_good_test
#model_good,dX_good_train

[23:17:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:17:13] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:17:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:17:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:17:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:17:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:17:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:17:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:17:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_

[23:18:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=4
[23:18:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:18:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=4
[23:18:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:18:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=4
[23:18:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=4
[23:18:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:18:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[23:18:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_

In [81]:
# 测试第一次训练结果
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
dgood_test = xgb.DMatrix(X_good_test)
y_good_pred = model_good.predict(dgood_test)
auc = roc_auc_score(y_true = y_good_test,y_score = y_good_pred)
print(auc)

0.8071945216926864


In [83]:
from collections import Counter
wrong = y_good_train[y_good_train!=y_train_pred]
print(Counter(wrong))
print(Counter(y_good_train))

Counter({0.0: 4559, 1.0: 1229})
Counter({0.0: 21900, 1.0: 1525})


In [82]:
y_train_pred[y_train_pred<0.5] = 0
y_train_pred[y_train_pred>=0.5] = 1

In [84]:
y_train_pred = model_good.predict(xgb.DMatrix(X_good_train))
auc1 = roc_auc_score(y_good_train,y_train_pred)
print(auc1)

0.961470125009357


In [9]:
# 用模型给好样本打分
dX_good_train = xgb.DMatrix(X_good_train)
weight_good = model_good.predict(dX_good_train)
region = list(np.linspace(0,1,11))
group_good = pd.cut(weight_good,region,right = False)
frequency_good  = np.ravel(group_good.codes)

percentage_good  = pd.Series(frequency_good).value_counts()
X_good_train['group'] = frequency_good
X_good_train['label'] = label

good_bad_rate = []
types = np.sort(percentage_good.index.values)
for i in types:
    select_good = X_good_train[X_good_train['group'] == i]
    total_amount = select_good.shape[0]
    try:
        amount = select_good['label'].value_counts()[1]
    except:
        amount = 0
    good_bad_rate.append((amount/total_amount)*5)
good_bad_rate = np.array(good_bad_rate)
good_bad_rate[good_bad_rate>1.0] = 1.0

#del select_good
del amount
gc.collect()
# 删除group, label column
X_good_train.drop(['group','label'],axis=1,inplace=True)

//anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
//anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
//anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

In [14]:
# 用模型给坏样本打分
dX_bad = xgb.DMatrix(X_bad)
y_bad_predict = model_good.predict(dX_bad)
group_bad = pd.cut(y_bad_predict,region,right = False)
frequency_bad = np.ravel(group_bad.codes)
X_bad['label'] = np.zeros(X_bad.shape[0])
X_bad['group'] = frequency_bad
for i in types:
    temp = X_bad[X_bad['group']==i]
    size = int(temp.shape[0] * good_bad_rate[i])
    if size>0:
        choice = np.random.choice(temp.index.values,size,replace=False)
        X_bad.loc[choice,'label'] = 1
        
label_bad = X_bad['label']
X_bad.drop(['group','label'],axis=1,inplace=True)

X_bad = X_bad[:8000]
label_bad = label_bad[:8000]

#可供选择
#label_bad

In [15]:
# 坏样本分出训练、测试集
X_bad_train, X_bad_test, y_bad_train, y_bad_test = train_test_split(X_bad,label_bad,
                                                                        test_size = 0.3,random_state = None)

# 整合数据，分为训练、测试集
X_train = pd.concat([X_good_train,X_bad_train],axis=0,ignore_index=True)
y_train = pd.concat([y_good_train,y_bad_train],axis=0,ignore_index=True)

X_test = pd.concat([X_good_test,X_bad_test],axis=0,ignore_index=True)
y_test = pd.concat([y_good_test,y_bad_test],axis=0,ignore_index=True)

# 可选数据
# X_test,y_test,X_train,y_train

In [16]:
# 训练整体样本
params = {
    'booster' : 'gbtree',
    'objective': 'binary:logistic',
    'gamma': 0.2, #minimum loss reduction required to make a further partition on a leaf node of the tree
    'max_depth': 5,
    'gamma': 2, #L1 penalty
    'subsample' : 0.8, #proportion of train set used to train
    'colsample_tree' : 1, #proportion of sample of features
    'min_child_weight': 1,
    'silent': 0, #print process
    'eta' : 0.01, # step size
    'scale_pos_weight': 10
}
plst= params.items()

dtrain = xgb.DMatrix(X_train,y_train)
num_rounds = 400
model = xgb.train(plst,dtrain,num_rounds)



[09:43:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:43:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:43:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:43:09] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:43:11] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:43:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:43:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:43:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:43:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_

[09:44:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[09:44:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 2 pruned nodes, max_depth=5
[09:44:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:44:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:44:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:44:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[09:44:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 2 pruned nodes, max_depth=5
[09:44:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:44:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_

[09:46:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[09:46:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[09:46:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[09:46:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:46:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[09:46:36] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[09:46:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[09:46:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:46:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_

[09:48:11] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[09:48:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[09:48:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[09:48:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 2 pruned nodes, max_depth=5
[09:48:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[09:48:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:48:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 2 pruned nodes, max_depth=5
[09:48:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:48:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 2 pruned nodes, max_

[09:49:49] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 2 pruned nodes, max_depth=5
[09:49:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[09:49:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[09:49:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[09:49:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_depth=5
[09:49:55] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 0 pruned nodes, max_depth=5
[09:49:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:49:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 2 pruned nodes, max_depth=5
[09:49:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, max_

[09:51:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[09:51:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 2 pruned nodes, max_depth=5
[09:51:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 6 pruned nodes, max_depth=5
[09:51:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[09:52:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 2 pruned nodes, max_depth=5
[09:52:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[09:52:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 2 pruned nodes, max_depth=5
[09:52:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 58 extra nodes, 2 pruned nodes, max_depth=5
[09:52:11] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 6 pruned nodes, max_

In [18]:
#检验parcelling 结果
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
dtest = xgb.DMatrix(X_test)
y_pred = model.predict(dtest)
auc_test = roc_auc_score(y_true = y_test,y_score = y_pred)
print(auc_test)

0.8704132131223982


# Valid Set

In [16]:
valid_set = pd.read_csv('open_data_train_valid/valid.txt',sep='\t')
numerical_data = valid_set[numerical_feature]
factor_data = valid_set[factor_feature]
numerical_data.fillna(numerical_data.mean(axis=0),axis=0,inplace = True)
#numerical_feat = scaler.transform(numerical_data)
#numerical_feat = pd.DataFrame(numerical_feat,index = numerical_data.index,columns=numerical_data.columns)
numerical_data = scaler.transform(numerical_data)
valid_data = pd.concat([numerical_data,factor_data],axis=1)
y_valid_pre = model_good.predict(valid_data)

//anaconda/lib/python3.6/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


TypeError: cannot concatenate a non-NDFrame object

In [65]:
y_valid_pre = model_good.predict(dvalid)

In [66]:
def write_file(filename,proba):
    with open(filename, 'w') as f:
        f.write('id,prob')
        f.write('\n')
        for i in range(proba.shape[0]):
            line = str(100000+i+1) + ',' + str(proba[i])
            f.write(line + '\n')

In [67]:
write_file('valid_sis.txt',y_valid_pre)

# fuzzy

In [50]:
dbad_train = xgb.DMatrix(X_bad_train)
weights = model_good.predict(dbad_train)
#读取labelled data

X_fuzzy_train = pd.concat([X_good_train,X_bad_train,X_bad_train],axis=0,ignore_index=True)

sample_weights = np.concatenate((np.ones(X_good_train.shape[0]),weights,1-weights),axis=0)
y_fuzzy_train = np.concatenate((y_good_train,np.ones(weights.shape[0]),np.zeros(weights.shape[0])),axis=0)
#定义新的clf
dfuzzy_train = xgb.DMatrix(X_fuzzy_train,y_fuzzy_train,weight=sample_weights)
params_fuzzy = {
    'booster' : 'gbtree',
    'objective': 'binary:logistic',
    'gamma': 0.1, #minimum loss reduction required to make a further partition on a leaf node of the tree
    'max_depth': 4,
    'gamma': 2, #L1 penalty
    'subsample' : 0.6, #proportion of train set used to train
    'colsample_tree' : 1, #proportion of sample of features
    'min_child_weight': 1,
    'silent': 0, #print process
    'eta' : 0.01, # step size
    'scale_pos_weight': 2
}
plst_fuzzy = params_fuzzy.items()
#clf_new.fit(X,Y_)
#xgb.train()

In [52]:
# 训练
num_rounds = 300
model_fuzzy = xgb.train(plst_fuzzy,dfuzzy_train,num_rounds)

[08:48:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:48:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[08:48:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:48:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[08:48:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[08:48:41] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:48:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:48:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:48:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 2 pruned nodes, max_

[08:49:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:49:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:49:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:49:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:49:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:49:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:49:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:49:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:49:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_

[08:49:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:49:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:49:29] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[08:49:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=4
[08:49:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:49:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:49:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[08:49:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[08:49:31] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_

In [62]:
y_fuzzy_pred = model_fuzzy.predict(dgood_test)
auc_fuzzy_test = roc_auc_score(y_true = y_good_test,y_score = y_fuzzy_pred)
print(auc_fuzzy_test)

0.7783971550996351


In [48]:
y_valid_pre = model_fuzzy.predict(dvalid)
write_file('valid_sis.txt',y_valid_pre)

In [34]:
type(X_train)

NameError: name 'X_train' is not defined